In [10]:
from openai import OpenAI
import os

OPENAI_API_KEY =" INSERT "
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [11]:
client = OpenAI()

In [12]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler): 

    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)
      
    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)
      
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
  
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
        if delta.code_interpreter.outputs:
            print(f"\n\noutput >", flush=True)
            for output in delta.code_interpreter.outputs:
                if output.type == "logs":
                    print(f"\n{output.logs}", flush=True)
                    self.outputs.append(output.logs)


In [ ]:
results = {}
for i in range(1,11):
    results[i] = {}
    for temp in [0,.1,.5]:
        MAX_RETRIES = 3

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                print("Attempting...")
                print('#######')
                print(i)
                print(temp)
                client = OpenAI()

                file = client.files.create(
                  file=open(f"experiment_Data/Case{i}.csv", "rb"),
                  purpose='assistants'
                )

                assistant = client.beta.assistants.create(temperature = temp,
                  instructions='''You are an endocrinologist analyzing CGM data for a patient with type 1 diabetes and on insulin therapy.
                Note that the  target glucose is between 70 mg/dL and 180 mg/dL, and severe hypoglycemia is defined as less than 54 mg/dL  and severe hyperglycemia is defined as greater than 250 mg/dL.
                Use the CGM data to answer the following questions''',
                  model="gpt-4-turbo-2024-04-09",
                  tools=[{"type": "code_interpreter"}],
                  tool_resources={
                    "code_interpreter": {
                      "file_ids": [file.id]
                    }
                  }
                )
                
                thread = client.beta.threads.create()
                message = client.beta.threads.messages.create(
                  thread_id=thread.id,
                  role="user",
                  content='''Report the following metrics: 
                1. Number of days the sensor is active
                2. % of sensor data captured
                3. Mean glucose concentration
                4. Glucose management indicator
                5. Glucose variability (coefficient of variation)
                6. % Time above target glucose range (TAR) in level 2 (>250)
                7. % Time above target glucose range (TAR) in level 1( >180)
                8. % Time in range (70-180 mg/dL)
                9. % Time below target glucose range (TBR) in level 1 (<70)
                10. % Time below target glucose range (TBR)  in level 2 (<54)

                Produce a list with these 10 values in it.
                '''
                )
                
                hand =EventHandler()
                with client.beta.threads.runs.stream(
                  thread_id=thread.id,
                  assistant_id=assistant.id,
                  instructions='''You are an endocrinologist analyzing CGM data for a patient with type 1 diabetes and on insulin therapy.
                Note that the  target glucose is between 70 mg/dL and 180 mg/dL, and severe hypoglycemia is defined as less than 54 mg/dL  and severe hyperglycemia is defined as greater than 250 mg/dL.
                Use the CGM data to answer the following questions''',
                  event_handler=hand,
                ) as stream:

                    stream.until_done()

                out = hand.current_message_snapshot.content[0].to_dict()
                result = out['text']['value']
                results[i][temp] = result
                print("Success! attempt no ", attempt)
                break  
            except Exception as e:
                print(f"Attempt {attempt} failed, error: {e}")
                if attempt == MAX_RETRIES:
                    print("attempts failed")
           